#구글 드라이브 및 기본 라이브러리 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os 
import cv2

import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import pickle
from sklearn.model_selection import train_test_split

from tqdm import tqdm

from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

from tensorflow.keras.callbacks import ModelCheckpoint

#이미지 불러오기

### 이미지 전처리
- RAM 용량 초과로 인해 일부 데이터만

In [ ]:
data_path = '/content/drive/MyDrive/팀플 1조 (객체인식) /수화이미지/수화'

In [ ]:
dataset = {}

for label in os.listdir(data_path):
    sub_path = '/content/drive/MyDrive/팀플 1조 (객체인식) /수화이미지/수화/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
      if filename[:1] == '1' :
        dataset[label].append(sub_path+filename)
      elif filename[:1] == '3' :
        dataset[label].append(sub_path+filename)
      elif filename[:1] == '5' :
        dataset[label].append(sub_path+filename)
      elif filename[:1] == '7' :
        dataset[label].append(sub_path+filename)

In [ ]:
x_datasets, y_datasets = [], []
label2index={}


for i,n in enumerate(os.listdir(data_path)) :
  label2index[n] = i
                

for label, filenames in tqdm(dataset.items()):
    for filename in filenames:
        image = cv2.imread(filename)
        resize = lambda x: tf.image.resize(x, (224, 224))
        image = resize(image)
        x_datasets.append(image)

        y_datasets.append(label2index[label])

###dataset array 변형

In [ ]:
x_datasets = np.array(x_datasets)
y_datasets = np.array(y_datasets)

###Grayscale

In [ ]:
def preprocess(image, grayscale=False):
    image -= np.mean(image, axis=0)
    
    if grayscale:
        image = tf.image.rgb_to_grayscale(image) 
    return np.array(image)

In [ ]:
x_datasets = preprocess(x_datasets, grayscale=True)

###Pickle 저장

In [ ]:
with open('drive/My Drive/x_dataset_1357.pickle', 'wb') as f:
    pickle.dump(x_datasets, f)

with open('drive/My Drive/y_dataset_1357.pickle', 'wb') as f:
    pickle.dump(y_datasets, f)

NameError: ignored

###Pickle 로드

In [ ]:
with open('/content/drive/MyDrive/팀플 1조 (객체인식) /피클 데이터/x1357합본.pickle', 'rb') as f:
    x_datasets = pickle.load(f)

with open('/content/drive/MyDrive/팀플 1조 (객체인식) /피클 데이터/y1357합본.pickle', 'rb') as f:
    y_datasets = pickle.load(f)

# Train, Test 세트 나누기

In [ ]:
# stratify 적용

train_images, test_images, train_labels, test_labels = train_test_split(x_datasets, y_datasets, test_size=0.3, random_state=1, stratify=y_datasets) 

In [ ]:
# 확인
train_images.shape, test_images.shape

((6189, 224, 224, 1), (2653, 224, 224, 1))

#모델 학습

####모델 학습을 위한 사전 준비

Shape 맞추기

In [ ]:
train_images = tf.repeat(train_images, 3, axis=3)
test_images = tf.repeat(test_images, 3, axis=3)
train_images.shape, test_images.shape

(TensorShape([6189, 224, 224, 3]), TensorShape([2653, 224, 224, 3]))

earlystop 선언

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

#####함수화

In [ ]:
def model():
    model = models.Sequential()

    model.add(layers.experimental.preprocessing.Resizing(224, 224, input_shape=train_images.shape[1:]))

    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Lambda(tf.nn.local_response_normalization))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same')) 
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Lambda(tf.nn.local_response_normalization))


    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same')) 
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))

    model.add(layers.Dense(40, activation='softmax')) 
    
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

원핫벡터화

In [ ]:
y_train = tf.keras.utils.to_categorical(train_labels, 40)
y_test = tf.keras.utils.to_categorical(test_labels, 40)
y_train.shape, y_test.shape

((6189, 40), (2653, 40))

###AlexNet
- 모델 간소화 버전

#####모델 가중치 저장

In [ ]:
# 체크포인트 경로 지정
AlexNet_cp_path = 'drive/My Drive/AlexNet/cp.ckpt'
AlexNet_checkpoint = ModelCheckpoint(filepath=AlexNet_cp_path,
                            save_best_only=True,
                            save_weights_only=True,
                            verbose=1)

#####AlexNet 학습

In [ ]:
AlexNet_model = model()

AlexNet_model.fit(train_images, train_labels, batch_size=150, epochs=5, validation_split=0.2, callbacks=[early, AlexNet_checkpoint])

Epoch 1/5
34/34 [==============================] - ETA: 0s - loss: 3.3870 - accuracy: 0.1553
Epoch 1: val_loss improved from inf to 2.64778, saving model to drive/My Drive/AlexNet/cp.ckpt
34/34 [==============================] - 266s 8s/step - loss: 3.3870 - accuracy: 0.1553 - val_loss: 2.6478 - val_accuracy: 0.3409
Epoch 2/5
34/34 [==============================] - ETA: 0s - loss: 1.5952 - accuracy: 0.5977
Epoch 2: val_loss improved from 2.64778 to 0.97124, saving model to drive/My Drive/AlexNet/cp.ckpt
34/34 [==============================] - 263s 8s/step - loss: 1.5952 - accuracy: 0.5977 - val_loss: 0.9712 - val_accuracy: 0.7601
Epoch 3/5
34/34 [==============================] - ETA: 0s - loss: 0.5544 - accuracy: 0.8544
Epoch 3: val_loss improved from 0.97124 to 0.48524, saving model to drive/My Drive/AlexNet/cp.ckpt
34/34 [==============================] - 264s 8s/step - loss: 0.5544 - accuracy: 0.8544 - val_loss: 0.4852 - val_accuracy: 0.8829
Epoch 4/5
34/34 [=====================

###전이학습

####분류기 부분만 재학습

In [ ]:
base_model = ResNet50(include_top=False, input_shape = (224,224 ,3), weights = 'imagenet')
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) 

x = tf.keras.layers.Flatten()(x) 
x = tf.keras.layers.Dense(256, activation='relu')(x) 
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(40, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 100352)            0         
                                                                 
 dense_4 (Dense)             (None, 256)               25690368  
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 40)                10280     
                                                                 
Total params: 49,288,360
Trainable params: 25,700,648
Non-t

######모델 가중치 저장

In [ ]:
ResNet1_cp_path = 'drive/My Drive/ResNet1/cp.ckpt'
ResNet1_checkpoint = ModelCheckpoint(filepath=ResNet1_cp_path,
                            save_best_only=True,
                            save_weights_only=True,
                            verbose=1)

######학습

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001), 
                  loss = 'categorical_crossentropy', 
                  metrics=['accuracy'])

In [ ]:
model.fit(train_images, y_train, epochs = 10, validation_data=(test_images, y_test), batch_size=150, callbacks=[early, ResNet1_checkpoint])

Epoch 1/10
42/42 [==============================] - ETA: 0s - loss: 4.9672 - accuracy: 0.2823 
Epoch 1: val_loss improved from inf to 1.70285, saving model to drive/My Drive/ResNet1/cp.ckpt
42/42 [==============================] - 1308s 31s/step - loss: 4.9672 - accuracy: 0.2823 - val_loss: 1.7029 - val_accuracy: 0.5511
Epoch 2/10
42/42 [==============================] - ETA: 0s - loss: 2.0404 - accuracy: 0.4388 
Epoch 2: val_loss improved from 1.70285 to 1.28942, saving model to drive/My Drive/ResNet1/cp.ckpt
42/42 [==============================] - 1304s 31s/step - loss: 2.0404 - accuracy: 0.4388 - val_loss: 1.2894 - val_accuracy: 0.6566
Epoch 3/10
42/42 [==============================] - ETA: 0s - loss: 1.7408 - accuracy: 0.5082 
Epoch 3: val_loss improved from 1.28942 to 1.06412, saving model to drive/My Drive/ResNet1/cp.ckpt
42/42 [==============================] - 1310s 31s/step - loss: 1.7408 - accuracy: 0.5082 - val_loss: 1.0641 - val_accuracy: 0.7041
Epoch 4/10
42/42 [========

####하위층 일부 재학습

In [ ]:
base_model = ResNet50(include_top=False, input_shape = (224,224 ,3), weights = 'imagenet')

base_model.trainable = False

for layer in base_model.layers[-50:]: 
  layer.trainable = True

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) 

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(40, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

#####모델 가중치 저장

In [ ]:
ResNet2_cp_path = 'drive/My Drive/ResNet2/cp.ckpt'
ResNet2_checkpoint = ModelCheckpoint(filepath=ResNet2_cp_path,
                            save_best_only=True,
                            save_weights_only=True,
                            verbose=1)

#####학습

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                  loss = 'categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
model.fit(train_images, y_train, epochs = 10, validation_data=(test_images, y_test), 
                   batch_size= 150, callbacks=[early, ResNet2_checkpoint])

Epoch 1/10
42/42 [==============================] - ETA: 0s - loss: 1.9976 - accuracy: 0.5398
Epoch 1: val_loss improved from inf to 0.41596, saving model to drive/My Drive/ResNet2/cp.ckpt
42/42 [==============================] - 88s 2s/step - loss: 1.9976 - accuracy: 0.5398 - val_loss: 0.4160 - val_accuracy: 0.8918
Epoch 2/10
42/42 [==============================] - ETA: 0s - loss: 0.5150 - accuracy: 0.8462
Epoch 2: val_loss improved from 0.41596 to 0.22041, saving model to drive/My Drive/ResNet2/cp.ckpt
42/42 [==============================] - 64s 2s/step - loss: 0.5150 - accuracy: 0.8462 - val_loss: 0.2204 - val_accuracy: 0.9450
Epoch 3/10
42/42 [==============================] - ETA: 0s - loss: 0.3127 - accuracy: 0.9115
Epoch 3: val_loss improved from 0.22041 to 0.17354, saving model to drive/My Drive/ResNet2/cp.ckpt
42/42 [==============================] - 64s 2s/step - loss: 0.3127 - accuracy: 0.9115 - val_loss: 0.1735 - val_accuracy: 0.9529
Epoch 4/10
42/42 [====================

####전부 재학습
- 오류로 인해서 학습 안하는 것으로

In [ ]:
base_model = ResNet50(include_top=False, input_shape = (224,224 ,3), weights = None)

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(40, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

#####모델 가중치 저장

In [ ]:
ResNet3_cp_path = 'drive/My Drive/ResNet3/cp.ckpt'
ResNet3_checkpoint = ModelCheckpoint(filepath=ResNet3_cp_path,
                            save_best_only=True,
                            save_weights_only=True,
                            verbose=1)

#####학습

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                  loss = 'categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
model.fit(train_images, y_train, epochs = 8, validation_data=(test_images, y_test), batch_size= 150, callbacks=[early, ResNet3_checkpoint])

####미세 조정

In [ ]:
base_model = ResNet50(include_top=False, input_shape = (224,224 ,3), weights = 'imagenet')

base_model.trainable = True

for layer in base_model.layers[-50:]: 
    layer.trainable = True				

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(40, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])

#####RAM 문제로 인해 fit이 되지 않아서 하위층 일부 재학습 체크포인트를 기준으로 평가

In [ ]:
model.load_weights(ResNet2_cp_path)

In [ ]:
model.evaluate(test_images, y_test, verbose=1)

83/83 [==============================] - 27s 281ms/step - loss: 0.0496 - accuracy: 0.9876


[0.049557361751794815, 0.9875612258911133]